# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: Ratas.py 🐁

- Nombre de alumno 1: Geraldyn Pérez
- Nombre de alumno 2: Diego Rojas

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Geral37/MDS7202.git)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

`escriba su respuesta acá`

El entorno Blackjack es un juego de cartas modelado como un problema de decisión secuencial, donde el objetivo es maximizar la recompensa jugando contra un crupier. El jugador debe tomar decisiones en cada estado basándose en su mano, la carta visible del crupier y la posibilidad de usar un as como 11. Este entorno se considera un Problema de Decisión de Markov (MDP) porque satisface la propiedad de memoria de Markov: el siguiente estado y recompensa dependen únicamente del estado actual y la acción tomada.

El espacio de acción está representado como un rango de valores discretos, {0, 1}, que indican las dos decisiones posibles del jugador. La acción 0 corresponde a plantarse ("Palo"), terminando el turno del jugador y permitiendo que el crupier actúe. Por otro lado, la acción 1 representa pedir una carta adicional ("Golpe"), lo que incrementa la suma de las cartas del jugador, con el riesgo de superar 21.

El espacio de observación está definido como una tupla de tres elementos: la suma actual de las cartas del jugador, el valor de la carta visible del crupier (un número entre 1 y 10, donde 1 representa un as) y un indicador binario que señala si el jugador tiene un as utilizable (es decir, si el as puede contar como 11 sin exceder 21). Este diseño permite al jugador tomar decisiones estratégicas basadas en el estado del juego.

Las recompensas en este entorno reflejan los resultados del juego: el jugador recibe +1 si gana, -1 si pierde y 0 en caso de empate. Si el jugador gana con un blackjack natural (un as y una carta de valor 10 como mano inicial), puede recibir una recompensa adicional de +1.5 si esta regla está habilitada. De lo contrario, la recompensa es de +1.

Un episodio termina cuando el jugador elige pedir carta y su mano supera 21, o cuando decide plantarse, momento en el cual el crupier juega y se determina el resultado final. Cabe destacar que un as siempre se considerará utilizable (como 11) a menos que hacerlo cause que el jugador supere 21. Esto asegura que el juego se ajuste a las reglas del blackjack estándar y facilite la implementación del entorno como un MDP.

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
total_rewards = [] # recompensas

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción aleatoria (el espacio de acción tiene 2 dimensiones: "acción 0" o "acción 1")
        action = env.action_space.sample()  # Esto selecciona una acción aleatoria

        # Tomar la acción en el entorno
        observation, reward, done, _, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -0.3746
Desviación estándar de las recompensas: 0.9048065207545755


Es mala perfomance porque la estrategia siempre entrega pérdida en valor esperado. El juego aleatorio estaría cargado en contra del jugador.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [ ]:
from stable_baselines3 import A2C

# Crear el modelo A2C con una red neuronal de 64 neuronas en una capa oculta
model = A2C("MlpPolicy", env, verbose=1)

# Entrenar el modelo (10000 pasos de entrenamiento)
model.learn(total_timesteps=10000)

# Evaluar el modelo en 10 episodios
for episode in range(10):
    obs, info = env.reset()  # Reiniciar el entorno
    done = False
    total_reward = 0
    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model.predict(obs)

        # Ejecutar la acción en el entorno
        obs, reward, done, truncated, info = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    print(f"Episodio {episode + 1} - Recompensa Total: {total_reward}")

# Guardar el modelo entrenado (opcional)
model.save("a2c_blackjack")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.14     |
|    ep_rew_mean        | -0.27    |
| time/                 |          |
|    fps                | 154      |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.287   |
|    explained_variance | 0.409    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.589    |
|    value_loss         | 0.423    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.03     |
|    ep_rew_mean        | -0.15    |
| time/                 |          |
|    fps                | 207      |
|    iterations         | 200      |
|    time_elapsed       | 4        |
|    total_timesteps    | 1000     |
| train/                |          |
|

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Cargar el modelo A2C entrenado (si lo tienes guardado)
#model = A2C.load("a2c_blackjack")

# Lista para almacenar las recompensas
total_rewards = []

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model.predict(observation)

        # Tomar la acción en el entorno
        observation, reward, done, truncated, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -0.1726
Desviación estándar de las recompensas: 0.957710415522354


Mantiene baja performance pero mejora respecto al baseline. Se obtiene un promedio de pérdidas menor.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [ ]:
env.reset()

(array([15, 10,  1]), {})

In [ ]:
# Función para obtener la acción del agente dado un estado
def obtener_accion(state):
    # Usamos el modelo entrenado para predecir la acción en base al estado
    action, _states = model.predict(state)
    return action

# Primer escenario: Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene un as
# Para simular este escenario, creamos un estado artificialmente. Suponemos que el estado es un vector de numpy.
# Nota: La representación del estado dependerá de cómo esté estructurado el entorno. Para fines de este ejercicio,
# asumimos que el estado es un array que contiene [suma_agente, suma_dealer, tiene_as].
# Este es un ejemplo simplificado.

estado_escenario_1 = np.array([6, 7, 0])  # Suma del agente es 6, dealer muestra 7, agente no tiene un as
accion_1 = obtener_accion(estado_escenario_1)

# Segundo escenario: Suma de cartas del agente es 19, dealer muestra un 3, agente tiene un as
estado_escenario_2 = np.array([19, 3, 1])  # Suma del agente es 19, dealer muestra 3, agente tiene un as
accion_2 = obtener_accion(estado_escenario_2)

# Imprimir las acciones en ambos escenarios
print(f"Acción en el escenario 1 (Agente suma 6, Dealer muestra 7, sin as): {accion_1}")
print(f"Acción en el escenario 2 (Agente suma 19, Dealer muestra 3, con as): {accion_2}")

Acción en el escenario 1 (Agente suma 6, Dealer muestra 7, sin as): 0
Acción en el escenario 2 (Agente suma 19, Dealer muestra 3, con as): 0


### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [ ]:
# Lista para almacenar las recompensas totales de cada episodio
rewards = []

# Ejecutar la simulación 10 veces con acciones aleatorias
for episode in range(10):
    obs, info = env.reset()  # Reiniciar el entorno
    done = False
    total_reward = 0

    while not done:
        # Seleccionar una acción aleatoria
        action = env.action_space.sample()

        # Tomar la acción en el entorno
        obs, reward, done, truncated, info = env.step(action)

        # Acumular la recompensa total
        total_reward += reward

    # Guardar la recompensa total del episodio
    rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
rewards = np.array(rewards)

# Calcular el promedio y desviación estándar de las recompensas
average_reward = np.mean(rewards)
std_deviation = np.std(rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -210.40957651492937
Desviación estándar de las recompensas: 156.8861580778027


Al igual que el baseline del punto anterior, en promedio se obtiene pérdidas al seguir esta estrategia.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [ ]:
from stable_baselines3 import TD3

# Crear el modelo A2C con una red neuronal de 64 neuronas en una capa oculta
model2 = TD3("MlpPolicy", env, verbose=1)

# Entrenar el modelo (10000 pasos de entrenamiento)
model2.learn(total_timesteps=10000)

# Evaluar el modelo en 10 episodios
for episode in range(10):
    obs, info = env.reset()  # Reiniciar el entorno
    done = False
    total_reward = 0
    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model2.predict(obs)

        # Ejecutar la acción en el entorno
        obs, reward, done, truncated, info = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    print(f"Episodio {episode + 1} - Recompensa Total: {total_reward}")

# Guardar el modelo entrenado (opcional)
model2.save("td2_LunarLander")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93.2     |
|    ep_rew_mean     | -721     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 186      |
|    time_elapsed    | 2        |
|    total_timesteps | 373      |
| train/             |          |
|    actor_loss      | 8.07     |
|    critic_loss     | 69       |
|    learning_rate   | 0.001    |
|    n_updates       | 272      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 93       |
|    ep_rew_mean     | -937     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 152      |
|    time_elapsed    | 4        |
|    total_timesteps | 744      |
| train/             |          |
|    actor_loss      | 19.6     |
|    critic_loss     

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [ ]:
# Cargar el modelo A2C entrenado (si lo tienes guardado)
#model2 = TD3.load("td2_LunarLander")

# Lista para almacenar las recompensas
total_rewards = []

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model2.predict(observation)

        # Tomar la acción en el entorno
        observation, reward, done, truncated, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Promedio de recompensas: -286.66797663417617
Desviación estándar de las recompensas: 139.41690965104962


#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [ ]:
# Crear el modelo con parámetros personalizados
model2 = TD3(
    "MlpPolicy",
    env,
    learning_rate=0.001,   # Ajusta la tasa de aprendizaje
    batch_size=16,         # Ajusta el tamaño de los lotes
    verbose=1              # Nivel de detalle en los logs
)

# Entrenar el modelo (10,000 pasos de entrenamiento)
model2.learn(total_timesteps=10000)

# Lista para almacenar las recompensas
total_rewards = []

# Repetir la simulación 5000 veces
for _ in range(5000):
    # Reiniciar el entorno
    observation, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Elegir una acción usando el modelo entrenado
        action, _states = model2.predict(observation)

        # Tomar la acción en el entorno
        observation, reward, done, truncated, _ = env.step(action)

        # Acumular la recompensa
        total_reward += reward

    # Guardar la recompensa total del episodio
    total_rewards.append(total_reward)

# Convertir la lista de recompensas a un array de numpy para análisis
total_rewards = np.array(total_rewards)

# Calcular el promedio y desviación estándar
average_reward = np.mean(total_rewards)
std_deviation = np.std(total_rewards)

# Reportar los resultados
print(f"Promedio de recompensas: {average_reward}")
print(f"Desviación estándar de las recompensas: {std_deviation}")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: IsLocked() == false

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [ ]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key: ··········
Enter your Tavily API key: ··········


### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [ ]:
%pip install --upgrade --quiet PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
import PyPDF2

doc_paths = ["prevencionycontrol.pdf", "ratasenpropiedad.pdf","ratasyratones.pdf"] 

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [ ]:
!pip install --upgrade --quiet faiss-cpu langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain_google_genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

# Inicializar el cargador de documentos y cargar cada documento
documents = []
for path in doc_paths:
    loader = PyPDFLoader(path)
    documents.extend(loader.load())  # Cargar documentos y añadirlos a la lista

# Inicializar el divisor de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(documents)  # Dividir documentos en chunks

# Inicializar embeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Crear vectorstore y vectorizar los documentos
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

# Mostrar un resumen del vectorstore
vectorstore



/usr/local/lib/python3.10/dist-packages/google/colab/html/_background_server.py:103: DeprecationWarning: make_current is deprecated; start the event loop first
  ioloop.make_current()


#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [60]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever_chain = retriever | format_docs


In [58]:
from langchain_core.prompts import PromptTemplate

rag_template = '''
Eres un asistente experto en el cuidado del hogar con respecto al control de roedores como ratas y ratones.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta útil:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

In [61]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)


La información proporcionada no contiene datos sobre "nulos y blancos".  Por lo tanto, no puedo responder a tu pregunta.



#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [62]:
questions = [
    "¿Cómo elegir una empresa de control de plagas adecuada?",
    "¿Qué pasa si se detectan ratas en la calle?",
    "¿De que prefieren alimentarse las ratas?"
]

# Iterar sobre las preguntas y obtener respuestas para cada una
for query in questions:
    response = rag_chain.invoke(query)  # Recupera la respuesta para cada pregunta
    print(f"Pregunta: {query}")
    print(f"Respuesta: {response}")
    print("\n")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pregunta: ¿Cómo elegir una empresa de control de plagas adecuada?
Respuesta: Para elegir una empresa de control de plagas adecuada, debe seguir estos pasos:

1. **Pregunte y entreviste a varias empresas:**  Obtenga información sobre sus servicios y experiencia.

2. **Pida referencias:** Consulte con vecinos y amigos para conocer sus experiencias con diferentes empresas.

3. **Busque información:** Revise directorios telefónicos e internet para encontrar empresas que ofrezcan "control integral de plagas".  Estas empresas suelen realizar revisiones, control, reparaciones y recomendaciones.



Pregunta: ¿Qué pasa si se detectan ratas en la calle?
Respuesta: Si se detectan ratas en la calle, la ASPB (se asume que es una agencia de control de plagas) actúa en la vía pública, la red de alcantarillado y otros espacios públicos como solares, equipamientos y mercados municipales.  Para reportarlo, se puede llamar al 010 o al 900 226 226, usar el Servicio de Atención en Línea, acudir a las Ofici

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [68]:
chunk_sizes = [100, 500, 1000]  # Tamaños de chunk a probar

for chunk_size in chunk_sizes:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=50)
    splits = text_splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)

    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    retriever_chain = retriever | format_docs
    rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)
    # Ejecutar la búsqueda y analizar las respuestas
    query = "¿Cómo controlar la población de ratas?"
    response = rag_chain.invoke(query)
    print(f"Respuesta con chunk_size {chunk_size}:\n{response}\n")


Respuesta con chunk_size 100:
Para controlar la población de ratas, primero debe buscar evidencia de su presencia.  Luego, controle la maleza, los matorrales y los arbustos, ya que las ratas suelen vivir en madrigueras.


Respuesta con chunk_size 500:
Para controlar la población de ratas, debe eliminar todo lo que necesitan para sobrevivir: comida, agua, refugio y rutas de movilidad.  Esto implica controlar la maleza, los matorrales y los arbustos, ya que las ratas suelen vivir en madrigueras debajo de ellos.  Se debe evitar el césped alto, arbustos, matorrales y mantillo cerca de los cimientos de las construcciones.  Es necesario quitar la hiedra alrededor de las madrigueras, mantener un espacio descubierto de 6 pulgadas alrededor de las construcciones y podar debajo de los arbustos.  Finalmente, se debe conservar espacio entre las plantas y evitar la vegetación densa.


Respuesta con chunk_size 1000:
Para controlar la población de ratas, es fundamental un manejo adecuado del medio am

In [69]:
k_values = [1, 3, 5, 10]  # Diferentes valores para k

for k in k_values:
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": k})
    retriever_chain = retriever | format_docs
    rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)
    # Ejecutar la búsqueda y analizar las respuestas
    query = "¿Cómo controlar la población de ratas?"
    response = rag_chain.invoke(query)
    print(f"Respuesta con k={k}:\n{response}\n")


Respuesta con k=1:
Para controlar la población de ratas, es necesaria una prevención activa.  Esto se debe a que la presencia de ratas causa incomodidad, daños materiales y molestias.  Aunque en los últimos años no hay evidencia de transmisión de enfermedades de roedores a personas en la ciudad, las ratas pueden ser portadoras de enfermedades transmisibles, representando un riesgo para la salud.


Respuesta con k=3:
Para controlar la población de ratas, es fundamental un manejo adecuado del medio ambiente, enfocándose en eliminar sus tres necesidades básicas: alimento, agua y refugio.  Esto implica prácticas de higiene tanto en el hogar como en el entorno.  Específicamente, se recomienda cosechar y recoger diariamente frutas y nueces maduras; guardar la comida sobrante de las mascotas por la noche; mantener podadas las palmeras y plantas de yuca, y eliminar o podar la hiedra y otras plantas densas lejos de estructuras; reparar fugas de agua y eliminar el agua estancada.  Adicionalmente

In [70]:
search_types = ["similarity", "similarity_score_threshold", "mmr"]  # Tipos de búsqueda a probar

for search_type in search_types:

    if search_type == "similarity_score_threshold":
        retriever = vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs={"k": 3, "score_threshold": 0.5}  # Ajusta score_threshold entre 0 y 1
        )
    else:
        retriever = vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs={"k": 3}
        )
    retriever_chain = retriever | format_docs
    rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

    # Ejecutar la búsqueda y analizar las respuestas
    query = "¿Cómo controlar la población de ratas?"
    response = rag_chain.invoke(query)
    print(f"Respuesta con search_type={search_type}:\n{response}\n")




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Respuesta con search_type=similarity:
Para controlar la población de ratas, es fundamental un manejo adecuado del medio ambiente, enfocándose en eliminar sus tres necesidades básicas: alimento, agua y refugio.  Esto implica prácticas de higiene tanto en el hogar como en el entorno.  Específicamente, se recomienda cosechar y recoger diariamente frutas y nueces maduras; guardar la comida sobrante de las mascotas por la noche; mantener podadas las palmeras y plantas de yuca, y eliminar o podar la hiedra y otras plantas densas lejos de estructuras; reparar fugas de agua y eliminar el agua estancada.  Adicionalmente, se debe controlar la maleza, matorrales y arbustos, manteniendo un espacio descubierto de 6 pulgadas alrededor de las construcciones, podando debajo de los arbustos, conservando espacio entre las plantas y evitando la vegetación densa.  Se debe mantener los jardines libres de maleza y basura.


Respuesta con search_type=similarity_score_threshold:
Para controlar la población de

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [72]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

react_prompt = hub.pull("hwchase17/react") # template de ReAct

search = TavilySearchResults(max_results = 1) # inicializamos tool
tools = [search]

agent = create_react_agent(llm, tools, react_prompt) # primero inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # lo transformamos a AgentExecutor para habilitar la ejecución de tools
response = agent_executor.invoke({"input": "¿Cómo controlar la población de ratas?"})
print(response["output"])

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


> Entering new AgentExecutor chain...
Thought: To answer this question effectively, I need to find reliable information on rat control methods.  A search engine like tavily_search_results_json will be helpful.

Action: tavily_search_results_json
Action Input: "effective rat control methods"
[{'url': 'https://effective-rat-control.cshelpjq.com/', 'content': 'Effective Rat Control 🛑 Nov 2024. effective rat control. mo

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [74]:
!pip install wikipedia

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=6ae70c0789b781ceaf064d304e1dd6e7529ef8b5be4719a1e22ad7d03e58e3e7
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [81]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Inicializar el agente ReAct
agent = create_react_agent(llm, tool, react_prompt)

# Convertir el agente a un AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=[tool], verbose=True)

# Ejecutar la consulta
response = agent_executor.run({"input": "¿Cómo controlar la población de ratas?"})

# Imprimir la respuesta
print(response["output"])


AttributeError: 'tuple' object has no attribute 'name'

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )